In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
import os
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)
import datetime
# from notebooks.notebook_helpers import *
from math import ceil
import time

# Define DAQ Settings

In [ ]:
trigger_board = '0001'

self_trigger_delay = 484

LED_pages = ["0x0000","0x0004","0x0008","0x000C"] # LED pages to cycle through
active_channels = "0x0001"
polarity = "0x4023" # "0x4023" # "0x0023" or "0x0027"
hostname = "192.168.2.3"
ssd_path = "/home/daq/ETROC2/ETROC-Data/"
# ssd_path = "/run/media/daq/T7/"

testing = False

if not testing:
    total_time = 16 * 60 * 60
    daq_time = 30
    run_number = "ET2p01_HPK_NH4_HV230_Offset20_Run1"
    # run_options = "--compressed_binary --suppress_fillers"
    run_options = "--compressed_binary --skip_translation"
else:
    total_time = 120
    daq_time = 30
    run_number = "test_3"
    run_options = "--compressed_binary --suppress_fillers "
    # run_options = "--compressed_binary --skip_translation"

# Run DAQ

In [ ]:
trigger_bit_delay = int(trigger_board+'11'+format(self_trigger_delay, '010b'), base=2)
process_time = daq_time + 5
iterations = ceil(total_time/process_time)
print(f'How many iteration? {iterations}')
LED_id = 0

for it in range(iterations):
    LED_page = LED_pages[0]
    reset_string = ""
    if(it==0): reset_string = "--check_valid_data_start"
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t {process_time} --run_name Run_{run_number} -o loop_{it} -v -w  -c 0 -p {polarity} -d {trigger_bit_delay} -a {active_channels} {run_options} --start_DAQ_pulse --stop_DAQ_pulse --ssd_path {ssd_path} {reset_string}".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Run_{run_number}_loop_{it}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit')
    while not IPC_queue.empty():
        pass
    time.sleep(daq_time)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser